In [ ]:
!python3 -m pip install google-adk==1.2.1

In [ ]:
!pip install crewai

In [ ]:
# main.py
from crewai import Crew
from agents  import schema_extractor_agent, object_metadata_agent, bq_mapper_agent
from tasks import AgentTasks
from langchain_core.utils.function_calling import convert_to_openai_tool
from tools.schema_listing_tool import list_bigquery_objects
from tools.object_metadata_tool import get_object_metadata

In [ ]:
# main.py
from crewai import Crew
from agents  import schema_extractor_agent, object_metadata_agent, bq_mapper_agent
from tasks import AgentTasks
from langchain_core.utils.function_calling import convert_to_openai_tool
from tools.schema_listing_tool import list_bigquery_objects
from tools.object_metadata_tool import get_object_metadata
def main():
    project_id='nine-quality-test'
    # Step 1: Get all object names
    # Call the raw Python function directly with parameters
    object_list = list_bigquery_objects.func(project_id=project_id)
    results = []
    for dataset in object_list.keys():
        all_objects = [
            ("table", name) for name in object_list[dataset]["tables"]
        ] + [
            ("view", name) for name in object_list[dataset]["views"]
        ]  # You can later include procedures/functions too

   

        for object_type, object_name in all_objects:
            print(f"\n🔍 Analyzing {object_type.upper()} `{object_name}`...")

            # Step 2: Metadata extraction task
      
            metadata_task = AgentTasks.create_metadata_task(project_id, dataset, object_type, object_name, object_metadata_agent)
            metadata_crew = Crew(
                agents=[object_metadata_agent],
                tasks=[metadata_task],
                verbose=False
            )
            
            metadata = metadata_crew.kickoff()         
            print('--------------')
            # Step 3: Mapping to BigQuery
            mapping_task = AgentTasks.create_mapping_task(metadata, bq_mapper_agent)
            mapping_crew = Crew(
                agents=[bq_mapper_agent],
                tasks=[mapping_task],
                verbose=False
            )
            bq_result = mapping_crew.kickoff()

            print(bq_result)
            print('------*************--------')
            results.append({
                "object": object_name,
                "type": object_type,
                "bigquery_schema": bq_result,
                "dataset":dataset
            })
            
            

    # Final report
    print("\n\n🧾 === BIGQUERY MIGRATION SUMMARY ===")
    for obj in results:
        print(f"\nObject: {obj['type'].upper()} `{obj['object']}`")
        print(obj["bigquery_schema"])

if __name__ == "__main__":
    main()


In [ ]:
if 1==1:
    project_id='nine-quality-test'
    # Step 1: Get all object names
    # Call the raw Python function directly with parameters
    object_list = list_bigquery_objects.func(project_id=project_id)
    results = []
    for dataset in object_list.keys():
        all_objects = [
            ("table", name) for name in object_list[dataset]["tables"]
        ] + [
            ("view", name) for name in object_list[dataset]["views"]
        ]  # You can later include procedures/functions too

   

        for object_type, object_name in all_objects:
            print(f"\n🔍 Analyzing {object_type.upper()} `{object_name}`...")

            # Step 2: Metadata extraction task
      
            metadata_task = AgentTasks.create_metadata_task(project_id, dataset, object_type, object_name, object_metadata_agent)
            metadata_crew = Crew(
                agents=[object_metadata_agent],
                tasks=[metadata_task],
                verbose=False
            )
            print(metadata_task.input)

In [ ]:
import os
from google.oauth2 import service_account
import vertexai
from langchain_google_vertexai import ChatVertexAI
import litellm

# Set environment variables
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "intelia-health-8e5f53efd3df.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "intelia-health"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

# Load credentials from service account file
credentials = service_account.Credentials.from_service_account_file(
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
)

# Initialize Vertex AI with credentials
vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
    credentials=credentials,
)

class ChatVertexAIWithCall(ChatVertexAI):
    def __init__(self, model, project, location, credentials, **kwargs):
        super().__init__(model=model, project=project, location=location, **kwargs)
        self._vertex_project = project
        self._vertex_location = location
        self._vertex_model = model
        self._vertex_credentials = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
        print('8888')

    def call(self, messages, **kwargs):
        print('*****you are here***')
        return litellm.completion(
            model=self._vertex_model,
            messages=messages,
            vertex_project=self._vertex_project,
            vertex_location=self._vertex_location,
            vertex_credentials=self._vertex_credentials,
            **kwargs,
        )


llm = ChatVertexAIWithCall(
    model="gemini-2.5-pro",
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
    credentials=credentials,
)


from crewai.llm import LLM


from crewai import Agent, Crew
from tools.object_metadata_tool import get_object_metadata

# Define your metadata_task somewhere here before using it
# Example:
# metadata_task = Task(...)
from crewai.llm import LLM

class LLMWrapper:
    def __init__(self, llm_instance):
        self.llm = llm_instance
    
    def generate(self, prompt, **kwargs):
        # The 'prompt' here should be converted into the messages format expected by your llm.call
        messages = [{"role": "user", "content": prompt}]
        # Call your llm.call and get the response
        response = self.llm.call(messages=messages, **kwargs)
        # Assuming response is a dict or object with 'text' or similar key - adjust if different
        # For litellm, the response might be just the text string, so just return it
        return response

    # If your Agent expects other methods, add them here as needed
llm = LLMWrapper(llm)
 

model =ChatVertexAIWithCall(
    model="gemini-2.5-pro",
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
    credentials=credentials,
)
model = ChatVertexAI(model="gemini-2.5-pro" ,   project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"])

object_metadata_agent = Agent(
    role="Object Metadata Retriever",
    goal="Get full metadata (columns, types) of a bigquery object",
    backstory="Knows how to introspect database objects in detail.",
    tools=[get_object_metadata],
    llm=model,
    verbose=True,
)

metadata_crew = Crew(
    agents=[object_metadata_agent],
    tasks=[metadata_task],  # Make sure this is defined!
    verbose=True,
)

metadata = metadata_crew.kickoff()
# print(metadata)


In [8]:
import os
from google.oauth2 import service_account
import vertexai
from langchain_google_vertexai import ChatVertexAI
import litellm

# Set environment variables
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "intelia-health-8e5f53efd3df.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "intelia-health"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

# Load credentials from service account file
credentials = service_account.Credentials.from_service_account_file(
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
)

# Initialize Vertex AI with credentials
vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
    credentials=credentials,
)
 

from crewai.llm import LLM


from crewai import Agent, Crew
from tools.object_metadata_tool import get_object_metadata

# Define your metadata_task somewhere here before using it
# Example:
# metadata_task = Task(...)
from crewai.llm import LLM
 
x = ChatVertexAI(model="gemini-2.5-pro" ,   project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"])

object_metadata_agent = Agent(
    role="Object Metadata Retriever",
    goal="Get full metadata (columns, types) of a bigquery object",
    backstory="Knows how to introspect database objects in detail.",
    tools=[get_object_metadata],
    llm=x,
    verbose=True,
)
print(type(object_metadata_agent.llm))
metadata_crew = Crew(
    agents=[object_metadata_agent],
    tasks=[metadata_task],  # Make sure this is defined!
    verbose=True,
)

metadata = metadata_crew.kickoff()
# print(metadata)


<class 'crewai.llm.LLM'>


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c82c5311-f00f-4641-a94e-9a2e7adfeb79                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Object Metadata Retriever                                                                               │
│                                                                                                                 │
│  Task: Get metadata for project nine-quality-test, dataset Fine_tunning and object `Contracts_train_sample`     │
│  from the bigquery.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=<agents.LLMWrapper object at 0x7faa3da293f0>                                                  │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 2df904ae-58d7-40a2-9632-acd8a0851911                                                                     │
│  Agent: Object Metadata Retriever                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: c82c5311-f00f-4641-a94e-9a2e7adfeb79                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=<agents.LLMWrapper object at 0x7faa3da293f0>
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [5]:
type(object_metadata_agent.llm)

crewai.llm.LLM

In [ ]:
dir(object_metadata_agent.llm.model)

In [ ]:
from google.auth import default
credentials, project = default()
print(f"Detected project: {project}")
print(f"Credentials type: {type(credentials)}")


In [ ]:
!pip uninstall opentelemetry-sdk opentelemetry-api opentelemetry-exporter-otlp-proto -y
!pip install langchain-openai
!pip install langchain
!pip install langchain[google-genai]
!pip install langchain[vertexai]
!pip install -U langchain langchain-google-vertexai google-cloud-aiplatform
!pip install pyodbc
!pip install pymssql



!pip install \
  opentelemetry-api==1.24.0 \
  opentelemetry-sdk==1.24.0 \
  opentelemetry-exporter-otlp==1.24.0
